# Imports

In [6]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

# Data

$ 
y0 = 3.4  x_{0} - 1.2 x_{2} \\
y1 = 2.7 x_{1} + 4.5
$

In [7]:
n_data = 20
n_f = 3
n_o = 2

x = np.random.uniform(-10, 10, (n_data, n_f))
w0 = np.array([[3.4, 0, -1.2], [0,2.7,0]])
b0 = np.array([0, 4.5])
t = np.dot(x, w0.T) + b0

x = torch.tensor(x)
t = torch.tensor(t)

# Creating a linear regression from scratch

In [8]:
class Linear_Regression:
    def __init__(self, n_features, n_outputs) -> None:
        self.n_f = n_features
        self.n_o = n_outputs
        self.w = torch.rand((self.n_o, self.n_f), dtype=torch.float64, requires_grad=True)
        self.b = torch.rand(self.n_o, dtype=torch.float64, requires_grad=True)

    def predict(self, x):
        return torch.matmul(x, self.w.T) + self.b

    def MSE(self, t, y):
        diff = t - y
        return torch.sum(diff * diff) / diff.numel()

    def train(self, x, t, n_epochs, lr):
        for e in range(n_epochs):
            y = self.predict(x)
            loss = self.MSE(t, y)
            loss.backward()
            with torch.no_grad():
                self.w -= self.w.grad * lr
                self.b -= self.b.grad * lr
                self.w.grad.zero_()
                self.b.grad.zero_()

    def show_weights(self):
        print("equation")

        for i in range(self.n_o):
            equation = "    y[{}] = ".format(i)
            for j in range(self.n_f):
                op = "+"
                if self.w[i][j] < 0:
                    op = "-"

                equation += " {}{:.4f} * x[{}]".format(op, torch.abs(self.w[i][j]), j)

            op = "+"
            if self.b[i] < 0:
                op = "-"
                
            
            equation += " {}{:.4f}".format(op, torch.abs(self.b[i]))
        
            print(equation)


In [137]:
LR = Linear_Regression(3, 2)
LR.train(x,t, 1000, 0.001)
LR.show_weights()

equation
    y[0] =  +3.4022 * x[0] -0.0010 * x[1] -1.2021 * x[2] +0.0634
    y[1] =  -0.0495 * x[0] +2.7224 * x[1] +0.0465 * x[2] +3.0915


# Linear Regression by PyTorch

In [42]:
train_ds = TensorDataset(x, t)
train_dl = DataLoader(train_ds, 5, shuffle=True)

In [43]:
for xs, ys in train_dl:
    print(xs)
    print(ys)

tensor([[-3.9081,  0.6804, -8.5044],
        [ 6.5973,  2.2366,  4.6929],
        [ 5.4197, -2.3071,  7.9853],
        [-2.9788, -5.4418, -3.0292],
        [-9.5052, -0.5815, -2.4114]], dtype=torch.float64)
tensor([[ -3.0824,   6.3371],
        [ 16.7993,  10.5387],
        [  8.8447,  -1.7292],
        [ -6.4928, -10.1928],
        [-29.4240,   2.9300]], dtype=torch.float64)
tensor([[ 3.5867,  7.9980, -5.8941],
        [-0.0957,  6.3610,  9.5457],
        [ 2.1498,  2.0023,  6.2305],
        [ 1.6127,  4.6527,  3.9627],
        [-4.4089, -1.2756, -0.2973]], dtype=torch.float64)
tensor([[ 19.2678,  26.0946],
        [-11.7802,  21.6746],
        [ -0.1672,   9.9061],
        [  0.7279,  17.0622],
        [-14.6335,   1.0559]], dtype=torch.float64)
tensor([[ 8.9400, -5.6465, -2.5049],
        [-4.2481,  2.7452, -1.2053],
        [-6.8211, -8.3035,  8.8300],
        [-5.7131, -7.7521, -2.1461],
        [-4.5491, -9.3307,  4.4210]], dtype=torch.float64)
tensor([[ 33.4020, -10.7455],
     

In [66]:
class Linear_Regression_PyTorch:
    def __init__(self, n_features, n_outputs) -> None:
        self.n_f = n_features
        self.n_o = n_outputs
        self.model = nn.Linear(self.n_f, self.n_o, dtype=torch.float64)


    def predict(self, x):
        return self.model(x)


    def train(self, x, t, n_epochs, batch_size, lr):
        train_ds = TensorDataset(x, t)
        train_dl = DataLoader(train_ds, batch_size, shuffle=True)
        opt = torch.optim.SGD(self.model.parameters(), lr=lr)

        for e in range(n_epochs):
            for xs, ts in train_dl:
                ys = self.model(xs)
                loss = torch.nn.functional.mse_loss(ys, ts)
                loss.backward()
                opt.step()
                opt.zero_grad()

            if (e + 1) % (n_epochs/10) == 0:
                print('Epoch [{}/{}], Loss: {:.4f}'.format(e+1, n_epochs, loss.item()))


    def show_weights(self):
        print("equation")
        w = self.model.weight
        b = self.model.bias

        for i in range(self.n_o):
            equation = "    y[{}] =".format(i)
            for j in range(self.n_f):
                op = "+"
                if w[i][j] < 0:
                    op = "-"

                equation += " {}{:.4f} * x[{}]".format(op, torch.abs(w[i][j]), j)

            op = "+"
            if b[i] < 0:
                op = "-"
                
            
            equation += " {}{:.4f}".format(op, torch.abs(b[i]))
        
            print(equation)

In [67]:
LRP = Linear_Regression_PyTorch(3,2)
LRP.train(x,t,1000,5,0.001)
LRP.show_weights()

Epoch [100/1000], Loss: 5.6574
Epoch [200/1000], Loss: 1.7916
Epoch [300/1000], Loss: 0.9735
Epoch [400/1000], Loss: 0.5780
Epoch [500/1000], Loss: 0.1458
Epoch [600/1000], Loss: 0.0889
Epoch [700/1000], Loss: 0.0676
Epoch [800/1000], Loss: 0.0206
Epoch [900/1000], Loss: 0.0071
Epoch [1000/1000], Loss: 0.0034
equation
    y[0] = +3.4002 * x[0] +0.0000 * x[1] -1.2001 * x[2] +0.0070
    y[1] = -0.0029 * x[0] +2.6994 * x[1] +0.0017 * x[2] +4.4073
